# Lecture 6 

Parts of this lecture will be homework, and I will post an updated version of it for the homework 6 after the lecture.

Topics that will be covered in the lecture and/or homework:
* Types of relations
* Blobs and External storage
* Computed and Imported Tables
* Part tables and simultaneous insert
* updating/editing settings
* alter a table definition
* adapted attributes
* renamed and projected inheritance

In [1]:
import datajoint as dj
import numpy as np
import pandas as pd

dj.conn()

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [2]:
# "frame" fetch format is experimental and may give unexpected errors
# we will use "array" as the standard fetch format.
dj.config['fetch_format'] = 'array'

In [3]:
schema = dj.schema('lecture6')

# reinitialize schema and drop all tables in schema, if it exists
schema.drop(True)
schema = dj.schema('lecture6') # drop and reinitialize it to remove all data and tables associated with it

## One-to-one dependencies

![alt-text](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f7/CPT-Databases-OnetoOne.svg/1024px-CPT-Databases-OnetoOne.svg.png)

In [ ]:
# Country
# one primary key: country_name

# Capital1
# inherits the primary keys of Country
# dependent attribute: capital_name

# Capital2
# capital_name is a primary key
# inherited the primary key of Country as a dependent attribute
# not a one-to-one dependency because one capital name can have multiple countries

In [4]:
@schema
class Country(dj.Manual):
    definition = """
    country_name : varchar(63)
    ---
    """
    
@schema
class Capital1(dj.Manual): # inherits primary key from Country
    definition = """
    -> Country
    ---
    capital_name : varchar(63)
    """
    
# This is not a one-to-one dependency - why?
# This is a one-to-many dependencies
@schema
class Capital2(dj.Manual):
    definition = """
    capital_name : varchar(63)
    ---
    -> Country
    """

In [5]:
Country.insert1({'country_name':'USA'})
Capital1.insert1({'country_name':'USA', 'capital_name':'Washington'})

In [6]:
Capital1()

country_name,capital_name
USA,Washington


In [7]:
Capital1.insert1({'country_name':'USA', 'capital_name':'New York'})
# country_name:USA (primary key) already exists, so error

DuplicateError: ("Duplicate entry 'USA' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [8]:
# can insert country_name:USA twice with multiple capital_name
Capital2.insert1({'country_name':'USA', 'capital_name':'Washington'})
Capital2.insert1({'country_name':'USA', 'capital_name':'New York'})

In [9]:
Capital2()

capital_name,country_name
New York,USA
Washington,USA


## One-to-many dependencies

![alt-text](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/CPT-Databases-OnetoMany2.svg/1024px-CPT-Databases-OnetoMany2.svg.png)

In [10]:
# -> means inherit
@schema
class Book(dj.Manual):
    definition = """
    book_name : varchar(122)
    ---
    author : varchar(122)
    """

# inherit the primary keys from Book
# chapter_name is also a primary key now
# no secondary columns in this case
@schema
class Chapter1(dj.Manual):
    definition = """
    -> Book
    chapter_name : varchar(122)
    ---
    """

# What is the difference? Inherit Book as secondary column
# they are both "one to many" but work differently
@schema
class Chapter2(dj.Manual):
    definition = """
    chapter_name : varchar(122)
    ---
    -> Book
    """

In [11]:
Book.insert1({'book_name':'Faust', 'author':'Goethe'})
Chapter1.insert1({'book_name':'Faust', 'chapter_name':'Part One'})
Chapter1.insert1({'book_name':'Faust', 'chapter_name':'Part Two'})
Chapter2.insert1({'book_name':'Faust', 'chapter_name':'Part One'})
Chapter2.insert1({'book_name':'Faust', 'chapter_name':'Part Two'})

In [12]:
Book.insert1({'book_name':'1984', 'author':'Orwell'})
Chapter1.insert1({'book_name':'1984', 'chapter_name':'Part One'})
Chapter1.insert1({'book_name':'1984', 'chapter_name':'Part Two'})

In [15]:
# A duplicate error creeps up again
# 'Part 1' and 'Part 2' already exist for a different Book
# not unique (can't be primary here)
Chapter2.insert1({'book_name':'1984', 'chapter_name':'Part One'})
Chapter2.insert1({'book_name':'1984', 'chapter_name':'Part Two'})

DuplicateError: ("Duplicate entry 'Part One' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

## Many-to-many dependencies
![alt-text](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/CPT-Databases-ManytoMany.svg/1024px-CPT-Databases-ManytoMany.svg.png)

### "Junction" table

![alt-text](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Databases-ManyToManyWJunction.jpg/1024px-Databases-ManyToManyWJunction.jpg)

In [13]:
# many to many dependencies
# why did I call this Book2? Book already exists
# Book2 has only one primary key (book_name)
@schema
class Book2(dj.Manual):
    definition = """
    book_name : varchar(122)
    ---
    """

# just one primary key as well (author)
@schema
class Author(dj.Manual):
    definition = """
    author : varchar(122)
    ---
    """

# junction table
# inherits primary keys from Author and Book2
@schema
class AuthorBook(dj.Manual):
    definition = """
    -> Author
    -> Book2
    ---
    """
    

In [14]:
Book2.insert1({'book_name':'Faust'})
Book2.insert1({'book_name':'Leiden des jungen Werther'})
Book2.insert1({'book_name':'1984'})

Author.insert1({'author':'Goethe'})
Author.insert1({'author':'Orwell'})

AuthorBook.insert1({'book_name':'Faust', 'author':'Goethe'})
AuthorBook.insert1({'book_name':'Leiden des jungen Werther', 'author':'Goethe'})
AuthorBook.insert1({'book_name':'1984', 'author':'Orwell'})

### Part tables

Another way to define a junction table is by using datajoint's master-part relations. These are not inherently different from defining normal dependencies, but they can help you organize your code and many datajoint-specific UIs do make a distinction how they work within the application. 

The master-part relationship ensures that all parts of a complex representation appear together or not at all. 

As a general rule, it is good to avoid using master-part relationships for `Manual` tables unless you want to build a many-to-many relation.

NB: You can assign multiple part tables to one master table.

Let's look at an example, to see what is happening here. 

In [15]:
# Part table and master table
# this is not recommended but can use it in specific ways
# build specific types of many-to-many relationships

# when adding an author, want to also link book
# must always inherit Author1 in the part table (primary keys)
# can inherit other things too (Book2)
@schema
class Author1(dj.Manual):
    definition = """
    author : varchar(122)
    ---
    """
    
    class AuthorBook(dj.Part):
        definition = """
        -> Book2
        -> Author1
        ---
        """
        
#    class AuthorKids(dj.Part): # just an example of another part
#        definition = """
#        -> Author1
#        name: varchar(31)
#        ---
#        """

In [16]:
Author1.insert1({'author':'Goethe'})
Author1.insert1({'author':'Orwell'})

# insert specifically into a part class (AuthorBook)
Author1.AuthorBook.insert1({'book_name':'Faust', 'author':'Goethe'})
Author1.AuthorBook.insert1({'book_name':'Leiden des jungen Werther', 'author':'Goethe'})
Author1.AuthorBook.insert1({'book_name':'1984', 'author':'Orwell'})

In [17]:
Author1()

author
Goethe
Orwell


In [18]:
Author1.AuthorBook() # joint table

book_name,author
Faust,Goethe
Leiden des jungen Werther,Goethe
1984,Orwell


#### <font color='crimson'>Fork-specific feature:</font> `insert1p`
In this fork, you can simultaneously insert into the master and part table using the `insert1p` method. This enforces simultaneous insertion, as intended by master-part relations. 
Let's look at some examples:

In [19]:
# this deletes all entries we just inserted
Author1.delete()

About to delete
`lecture6`.`author1__author_book`: 3 items
`lecture6`.`author1`: 2 items
Proceed? [yes, No]: yes
Commited.


In [20]:
# insert into master and part at the same time (insert1p)
goethe_books = pd.DataFrame({'book_name': ['Faust', 'Leiden des jungen Werther']})
orwell_books = pd.DataFrame({'book_name': ['1984']})

In [21]:
Author1.insert1p(
    {'author': 'Goethe', 'AuthorBook': goethe_books}
)

Author1.insert1p(
    {'author': 'Orwell', 'AuthorBook': orwell_books}
)

In [22]:
Author1.AuthorBook()

book_name,author
Faust,Goethe
Leiden des jungen Werther,Goethe
1984,Orwell


In [23]:
# this deletes all entries we just inserted
Author1.delete()

About to delete
`lecture6`.`author1__author_book`: 3 items
`lecture6`.`author1`: 2 items
Proceed? [yes, No]: yes
Commited.


In [24]:
all_books = [
    {'author': 'Goethe', 'AuthorBook': goethe_books}, 
    {'author': 'Orwell', 'AuthorBook': orwell_books}
]

In [25]:
# use with caution (very experimental)
Author1().insertp(all_books)

In [26]:
Author1.AuthorBook()

book_name,author
Faust,Goethe
Leiden des jungen Werther,Goethe
1984,Orwell


<font color='crimson'>Caution</font>: `insert1p` and `insertp` are experimental features that will likely be adjusted in future main releases.

NB: A better approach for dealing with many-to-many relations will likely be implemented in the future.

## Advanced datatypes

Sometimes you want a column to not contain simple datatypes, such as string, integers or float, but you actually want the datatype to be a numpy.ndarray, a dictionary, or something else. 

### Blob datatype

The numpy.ndarray datatype is always supported, but in order to use python native datatypes (e.g. dicts and lists) you will need to enable them in your configuration file:

In [28]:
dj.config['enable_python_native_blobs'] = True
# allows you to insert dictionaries (and others) into the main blob datatype

#### <font color='crimson'>Fork-specific feature:</font> Pickling of data
If you want to store arbitrary python datatypes (either your custom class or for example pandas Dataframe), you will be able to enable pickling in the fork that we installed. 

I briefly mentioned in the previous lecture. Pickle is a standard python module that allows you to serialize and de-serialize any Python object. If you are pickling your own custom class, pickling can get a little messy over time unless you now what you are doing. In those cases, it is recommended to save your classes by first converting them into a native python datatype or a more standard type such as numpy.ndarray and pandas dataframes.

In [29]:
dj.config['enable_python_pickle_blobs'] = True
# pickling allows you to serialize or deserialize

Let's start using the blob datatype. Importantly, blob datatypes cannot be primary keys. 

In [30]:
# auto_increment automatically uses the next # (primary key) that's available
# the first number will be 1 and not 0
# longblob is the biggest version of blob datatype
@schema
class BlobTable(dj.Manual):
    definition = """
    id : int auto_increment
    ---
    data : longblob
    """

In [31]:
table = BlobTable()
table.insert1({'data':np.random.random((5, 6))})
# numpy array of size 5,6

In [33]:
# native datatype
table.insert1({'data':{'test':'this', 'with':'this'}})
# datatype that can be pickled
table.insert1({'data':pd.DataFrame()}) # empty data frame
# cannot do this without fork pickling enabled

In [34]:
table

id,data
1,=BLOB=
2,=BLOB=
3,=BLOB=
4,=BLOB=


In [35]:
pd.DataFrame(table.fetch())

,id,data
0,1,"[[0.8106109827341849, 0.43576139021576243, 0.9..."
1,2,"{'test': 'this', 'with': 'this'}"
2,3,"{'test': 'this', 'with': 'this'}"
3,4,Empty DataFrame Columns: [] Index: []


Currently, there is no direct way in the definition to restrict the datatype per column. This is a feature that will be implemented in the future. However, you can use custom/adapted attributes, which we will touch upon later and in the homework. 

### External storage

Here for more detail: https://docs.datajoint.io/python/definition/06.5-External-Data.html

Sometimes you don't want to store the data in the actual fields in the MySQL database, but you want to store them externally and keep track of those data via a link in the mysql database. In this case, you can configure an external storage system.

In general, it is recommended (for performance reasons) to use an external storage system if data for one field generally exceeds 10MB. 

Let's configure our external stores, and take a look at an example:

In [36]:
# we will have one external store in our current working directory
# you can define multiple stores and there are different types (see the link above for more info)
import os
folder = os.getcwd()
dj.config['stores'] = {
    'localstore': {
        'protocol': 'file', # store is a normal filepath
        'location': os.path.join(folder, 'localstore') # location of store
    }
}

In [37]:
# this is my current working directory
folder

'/Users/skb/Documents/PythonCourse2020/PythonDataCourse/Lectures/Lecture 6'

In [38]:
@schema
class BlobTable2(dj.Manual):
    definition = """
    id : int auto_increment
    ---
    data : blob@localstore # external storage
    """
#blob@localstore: blob is the datatype and localstore is the external storage system

In [39]:
table = BlobTable2()
table.insert1({'data':np.random.random((5, 6))})

In [40]:
# native datatype
table.insert1({'data':{'test':'this', 'with':'this'}})
# datatype that can be pickled
table.insert1({'data':pd.DataFrame()})

In [41]:
table

id,data external storage
2,=BLOB=
3,=BLOB=
1,=BLOB=


In [42]:
pd.DataFrame(table.fetch())

,id,data
0,2,"{'test': 'this', 'with': 'this'}"
1,3,Empty DataFrame Columns: [] Index: []
2,1,"[[0.07592645228471351, 0.6054712301725681, 0.0..."


Let's take a look inside our localstore folder

### Attach datatype

This works similar to your blob external store, but instead of storing a Python datatype, it will store a linked file in your external storage. This can be useful, when you want to store arbitrary files, such as jpg or zip files.

Let's take a look at an example:

In [43]:
# store using the attach datatype
dj.config['stores'].update({
    'attachstore': {
        'protocol': 'file', # store is a normal filepath
        'location': os.path.join(folder, 'attachstore') # location of store
    }
})

In [44]:
# attach datatype, attach storage system
@schema
class AttachTable(dj.Manual):
    definition = """
    id : int auto_increment  # what is this?
    ---
    data : attach@attachstore 
    """

In [45]:
# want to store a particular file
AttachTable.insert1({'data':os.path.join('images', 'some_image.png')})

In [46]:
AttachTable()

id what is this?,data
1,=BLOB=


In [47]:
AttachTable.fetch() 
# copy over into local directory
# this will fetch the data and copy the file saved in attachstore into your local directory

array([(1, 'some_image.png')], dtype=[('id', '<i8'), ('data', 'O')])

In [54]:
# you can also specify the download path for attachments
AttachTable.fetch(download_path='downloaded_attachments') 

array([(1, 'downloaded_attachments/some_image.png')],
      dtype=[('id', '<i8'), ('data', 'O')])

#### <font color='crimson'>Fork-specific feature:</font> Default download path

You can set the default download path in the config:

In [55]:
dj.config['tmp_folder'] = 'downloaded_attachments2'

In [56]:
AttachTable.fetch() 

array([(1, 'downloaded_attachments2/some_image.png')],
      dtype=[('id', '<i8'), ('data', 'O')])

#### Question: Why would the attachement be downloaded and not simply point to the file in the datajoint-configured store?

### Filepath datatype (preview/experimental)

This datatype works similar to the attach datatype, except that you it links to files already managed outside of datajoint.

Going through filepath datatypes will be part of the homework.

### Custom attributes - adapted attributes (preview/experimental)

This is a neat feature that allows you to define your own attributes. 

`AttributeAdapter` is a new experimental feature since version 0.12 that allows storing more complex objects. Since it's experimental, it's not yet included in the main documentation. To enable it, please set the environment variable DJ_SUPPORT_ADAPTED_TYPES to "TRUE":

In [57]:
# enable the use of adapted types for this session. 
# Adapted types are disabled by default since the feature is considered experimental.
import os
os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = "TRUE"  

We will go through one quick example of how to use `AttributeAdapter`. Let's say we want to save a short list of tags into the database. We could save it is a blob, but that doesn't restrict it to a list type plus it cannot be viewed in the database view (it will just say blob before fetching). We could implement a tag list by adapting the varchar string type:

In [58]:
import json

# subclass AttributeAdapter
class ListString(dj.AttributeAdapter):
    
    # this attribute is required (1)
    attribute_type = 'varchar(4000)'

    # this method is required (2)
    def put(self, obj):
        # if None return None (to allow nullable)
        if obj is None:
            return
        # check if obj is a list or tuple
        assert isinstance(obj, (list, tuple)), \
            f'object must be list or tuple for liststring type: {type(obj)}'
        # use json.dumps to dump the list as a string for insertion
        return json.dumps(obj)

    # this method is required (3)
    def get(self, value):
        # load the string and convert it to a list using json.loads
        return json.loads(value) # return as a list

In [59]:
liststring = ListString()

In [60]:
@schema
class TagTable(dj.Manual):
    definition = """
    id : int auto_increment  # what is this?
    ---
    tags : <liststring>
    """

In [61]:
TagTable.insert1({'tags':['asdf', 'adf', 'fsjkf']})
# it is a list, not a string even though the inserted data is a string

In [62]:
TagTable()

id what is this?,tags
1,"['asdf', 'adf', 'fsjkf']"


In [63]:
pd.DataFrame(TagTable.fetch())

,id,tags
0,1,"[asdf, adf, fsjkf]"


You could also use `AttributeAdapter` to do type checking and simply restrict the blob datatype to lists. 

#### NB

Do not use adapted attributes as primary keys, since they do not work once inherited in a "child" table. 

Your attribute must be in the context of the table that uses it (i.e. either it is defined in the same file or you import the instance into file that has the table). 

#### <font color='crimson'>Fork-specific feature:</font> Backup context for adapted attributes

Another EXPERIMENTAL option is to set all your adapted attributes in your config. You cannot save this to your datajoint config file - make sure to pop it! This feature will be handled more effectively in future versions. 

In [65]:
dj.config['backup_context'] = {'liststring': ListString()}

In [66]:
# delete liststring instance from context
del(liststring)

In [67]:
# this will use the backup_context
TagTable.insert1({'tags':['asf', 'adf', 'fsjkf']})

In [68]:
TagTable.fetch()

array([(1, list(['asdf', 'adf', 'fsjkf'])),
       (2, list(['asf', 'adf', 'fsjkf']))],
      dtype=[('id', '<i8'), ('tags', 'O')])

## Imported and Computed tables
Read this for more info: https://docs.datajoint.io/python/computation/Computation.html

In datajoint, `Imported` and `Computed` tables work exactly the same. The main purpose of these types of tables is for you to autopopulate them using custom-defined functions. You don't

They each require you to define a `make` method that tells the table how it should `populate` the table. 

Before we start let's import our `database_pipeline.py` file and take a look inside.

In [69]:
import database_pipeline as db

### `make` method

For auto-populated tables, data should never be entered using `insert` directly. Instead these tables must define the callback method `make(self, key)`. The insert method then can only be called on self inside this callback method.

Let's take a look inside our `database_pipline.py` file:

```python
@schema
class ImportedRecording(dj.Imported):
    definition = """
    -> RecordingSession
    ---
    data : blob@localstore
    """
    
    def make(self, key):
        filename = (RecordingSession & key).fetch1()['filename']
        data = pd.read_csv(os.path.join('data', filename))
        
        # key is a dictionary
        key.update({'data':data})
        
        self.insert1(key)
```

### `populate` method

The inherited `populate` method of `Imported` and `Computed` automatically calls `make` for every key for which the auto-populated table is missing data.

Let's populate our `Imported` and `Computed` table from our pipeline:

In [70]:
# first insert data into the manual table
insert_data = pd.read_csv('insert_data.csv')
print(insert_data)
db.RecordingSession.insert(insert_data.to_dict('records'))
print(db.RecordingSession())

   recording_id filename experimenter recording_quality
0             1        a           me              good
1             2        b           me              good
2             3        c           me              good
3             4        d           me              good
4             5        e           me              good
5             6        f           me              good
6             7        g           me              good
7             8        h           me              good
8             9        i           me              good
9            10        j           me              good
*recording_id  filename     experimenter   recording_qual comments    
+------------+ +----------+ +------------+ +------------+ +----------+
1              a            me             good           None        
2              b            me             good           None        
3              c            me             good           None        
4              d            m

In [71]:
# let's populate the imported table
db.ImportedRecording.populate()

{'recording_id': 1}
{'recording_id': 2}
{'recording_id': 3}
{'recording_id': 4}
{'recording_id': 5}
{'recording_id': 6}
{'recording_id': 7}
{'recording_id': 8}
{'recording_id': 9}
{'recording_id': 10}


In [60]:
db.ImportedRecording()

recording_id,data
4,=BLOB=
6,=BLOB=
8,=BLOB=
7,=BLOB=
10,=BLOB=
3,=BLOB=
5,=BLOB=
2,=BLOB=
1,=BLOB=
9,=BLOB=


What are some populate options?

In [72]:
table = db.ImportedRecording()
table.populate?
# pop-up window

In [73]:
db.AnalyzedRecording.populate('recording_id < 4')

{'recording_id': 1}
{'recording_id': 2}
{'recording_id': 3}


In [74]:
db.AnalyzedRecording()

recording_id,mean
1,=BLOB=
2,=BLOB=
3,=BLOB=


In [75]:
# let's insert more data into the recordingsession
db.RecordingSession.insert1({
    'recording_id': 11, 
    'filename': 'a', 
    'experimenter': 'me', 
    'recording_quality': 'ugly'
})

In [76]:
db.AnalyzedRecording.populate()

{'recording_id': 8}
{'recording_id': 4}
{'recording_id': 6}
{'recording_id': 9}
{'recording_id': 10}
{'recording_id': 7}
{'recording_id': 5}


In [77]:
db.AnalyzedRecording()

recording_id,mean
1,=BLOB=
2,=BLOB=
3,=BLOB=
4,=BLOB=
5,=BLOB=
6,=BLOB=
7,=BLOB=
8,=BLOB=
9,=BLOB=
10,=BLOB=


#### Question: Why did it not populate recording_id 11?

In [65]:
db.ImportedRecording.populate()
db.AnalyzedRecording.populate()

{'recording_id': 11}
{'recording_id': 11}


#### <font color='crimson'>Fork-specific feature:</font> Simple parallel computation

One of the options in the `populate` method is multiprocess. This allows you to perform parallel insertion into the database using multiple cpus. This feature will be part of the main release in the future, but can be used in this fork already

### Master-Part relationship
read more: https://docs.datajoint.io/python/computation/03-master-part.html

We will go through an example of using master-part relations in `Computed` tables for the homework.

## Altering tables

Going through altering tables will be part of the homework.

## Updating/editing existing entries

Going through updating/editing existing entries will be part of the homework.